## Sending the test event to Event Hub

In [0]:
from azure.eventhub import EventHubProducerClient, EventHubConsumerClient, EventData
import json

# Event Hub Configuration
EVENT_HUB_CONNECTION_STRING = dbutils.secrets.get(scope="key-vault-secret-scope", key="eventhub-connection-string")
EVENT_HUB_NAME = "weatherstreamingeventhub"

# Initialize the Event Hub Producer
producer = EventHubProducerClient.from_connection_string(
    conn_str=EVENT_HUB_CONNECTION_STRING,
    eventhub_name=EVENT_HUB_NAME
)
# Function to send events to Event Hub
def send_events(events):
    event_data_batch = producer.create_batch()
    event_data_batch.add(EventData(json.dumps(events)))
    producer.send_batch(event_data_batch)

# Sample JSON event
event = {
    "event_id": 2222,
    "event_name": "Key Vault Event"
}

# Send the event to Event Hub
send_events(event)

# Close the producer
producer.close()


## API Testing

In [0]:
import requests
import json

# Getting the secret value from Key Vault
weatherapikey = dbutils.secrets.get(scope="key-vault-secret-scope", key="weatherapikey")
location = "Regina"

base_url = "http://api.weatherapi.com/v1/"
current_weather_url = f"{base_url}/current.json"
forecast_weather_url = f"{base_url}/forecast.json."

params = {
    "key": weatherapikey,
    "q": location,
}
response = requests.get(current_weather_url, params=params)

if response.status_code == 200:
  current_weather = response.json()
  print("Current Weather: ")
  print(json.dumps(current_weather, indent=3))
else:
  print(f"Error: {response.status_code} - {response.text}")


Current Weather: 
{
   "location": {
      "name": "Regina",
      "region": "Saskatchewan",
      "country": "Canada",
      "lat": 50.45,
      "lon": -104.6167,
      "tz_id": "America/Regina",
      "localtime_epoch": 1758148956,
      "localtime": "2025-09-17 16:42"
   },
   "current": {
      "last_updated_epoch": 1758148200,
      "last_updated": "2025-09-17 16:30",
      "temp_c": 18.3,
      "temp_f": 64.9,
      "is_day": 1,
      "condition": {
         "text": "Overcast",
         "icon": "//cdn.weatherapi.com/weather/64x64/day/122.png",
         "code": 1009
      },
      "wind_mph": 7.4,
      "wind_kph": 11.9,
      "wind_degree": 99,
      "wind_dir": "E",
      "pressure_mb": 1023.0,
      "pressure_in": 30.2,
      "precip_mm": 0.0,
      "precip_in": 0.0,
      "humidity": 68,
      "cloud": 75,
      "feelslike_c": 18.3,
      "feelslike_f": 64.9,
      "windchill_c": 19.0,
      "windchill_f": 66.1,
      "heatindex_c": 19.0,
      "heatindex_f": 66.2,
      "dewp

## Complete Code for getting weather data

In [0]:
import requests
import json
from datetime import datetime, timedelta

# Function to handle the API response
def handle_response(response):
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code}, {response.text}"

# Function to get current weather and air quality data
def get_current_weather(base_url, api_key, location):
    current_weather_url = f"{base_url}/current.json"
    params = {
        'key': api_key,
        'q': location,
        "aqi": 'yes'
    }
    response = requests.get(current_weather_url, params=params)
    return handle_response(response)

# Function to get historical weather and air quality data
def get_history_weather(base_url, api_key, location, date):
    history_weather_url = f"{base_url}/history.json"
    params = {
        'key': api_key,
        'q': location,
        'dt': date,
        "aqi": 'yes'
    }
    response = requests.get(history_weather_url, params=params)
    return handle_response(response)

# Function to get Forecast Data
def get_forecast_weather(base_url, api_key, location, days):
    forecast_url = f"{base_url}/forecast.json"
    params = {
        "key": api_key,
        "q": location,
        "days": days,
    }
    response = requests.get(forecast_url, params=params)
    return handle_response(response)

# Function to get Alerts
def get_alerts(base_url, api_key, location):
    alerts_url = f"{base_url}/alerts.json"
    params = {
        'key': api_key,
        'q': location,
        "alerts": 'yes'
    }
    response = requests.get(alerts_url, params=params)
    return handle_response(response)

# Flatten and merge the data
def flatten_data(current_weather, forecast_weather, alerts, history_weather_list=None):
    location_data = current_weather.get("location", {})
    current = current_weather.get("current", {})
    condition = current.get("condition", {})
    air_quality = current.get("air_quality", {})
    forecast = forecast_weather.get("forecast", {}).get("forecastday", [])
    alert_list = alerts.get("alerts", {}).get("alert", [])
    history = []

    # Collect history from multiple days
    if history_weather_list:
        for hw in history_weather_list:
            days = hw.get("forecast", {}).get("forecastday", [])
            for day in days:
                history.append({
                    'date': day.get('date'),
                    'maxtemp_c': day.get('day', {}).get('maxtemp_c'),
                    'mintemp_c': day.get('day', {}).get('mintemp_c'),
                    'condition': day.get('day', {}).get('condition', {}).get('text')
                })

    flattened_data = {
        'name': location_data.get('name'),
        'region': location_data.get('region'),
        'country': location_data.get('country'),
        'lat': location_data.get('lat'),
        'lon': location_data.get('lon'),
        'localtime': location_data.get('localtime'),
        'temp_c': current.get('temp_c'),
        'is_day': current.get('is_day'),
        'condition_text': condition.get('text'),
        'condition_icon': condition.get('icon'),
        'wind_kph': current.get('wind_kph'),
        'wind_degree': current.get('wind_degree'),
        'wind_dir': current.get('wind_dir'),
        'pressure_in': current.get('pressure_in'),
        'precip_in': current.get('precip_in'),
        'humidity': current.get('humidity'),
        'cloud': current.get('cloud'),
        'feelslike_c': current.get('feelslike_c'),
        'uv': current.get('uv'),
        'air_quality': {
            'co': air_quality.get('co'),
            'no2': air_quality.get('no2'),
            'o3': air_quality.get('o3'),
            'so2': air_quality.get('so2'),
            'pm2_5': air_quality.get('pm2_5'),
            'pm10': air_quality.get('pm10'),
            'us-epa-index': air_quality.get('us-epa-index'),
            'gb-defra-index': air_quality.get('gb-defra-index')
        },
        'alerts': [
            {
                'headline': alert.get('headline'),
                'severity': alert.get('severity'),
                'description': alert.get('desc'),
                'instruction': alert.get('instruction')
            }
            for alert in alert_list
        ],
        'forecast': [
            {
                'date': day.get('date'),
                'maxtemp_c': day.get('day', {}).get('maxtemp_c'),
                'mintemp_c': day.get('day', {}).get('mintemp_c'),
                'condition': day.get('day', {}).get('condition', {}).get('text')
            }
            for day in forecast
        ],
        'history': history
    }
    return flattened_data

# Main program
def fetch_weather_data():
    base_url = "http://api.weatherapi.com/v1"
    location = "Regina"  # Replace with your city
    weatherapikey = dbutils.secrets.get(scope="key-vault-secret-scope", key="weatherapikey")

    # Collect last 3 days history
    history_weather_list = []
    for i in range(1, 8):  # last 7 days
        date = (datetime.today() - timedelta(days=i)).strftime("%Y-%m-%d")
        hw = get_history_weather(base_url, weatherapikey, location, date)
        history_weather_list.append(hw)

    # Get data from API
    current_weather = get_current_weather(base_url, weatherapikey, location)
    forecast_weather = get_forecast_weather(base_url, weatherapikey, location, 7)
    alerts = get_alerts(base_url, weatherapikey, location)

    # Flatten and merge data
    merged_data = flatten_data(current_weather, forecast_weather, alerts, history_weather_list)
    print("Weather Data:", json.dumps(merged_data, indent=3))

# Calling the Main Program
fetch_weather_data()

Weather Data: {
   "name": "Regina",
   "region": "Saskatchewan",
   "country": "Canada",
   "lat": 50.45,
   "lon": -104.6167,
   "localtime": "2025-09-17 17:06",
   "temp_c": 18.0,
   "is_day": 1,
   "condition_text": "Overcast",
   "condition_icon": "//cdn.weatherapi.com/weather/64x64/day/122.png",
   "wind_kph": 11.2,
   "wind_degree": 88,
   "wind_dir": "E",
   "pressure_in": 30.2,
   "precip_in": 0.0,
   "humidity": 68,
   "cloud": 75,
   "feelslike_c": 18.0,
   "uv": 0.8,
   "air_quality": {
      "co": 349.65,
      "no2": 15.355,
      "o3": 50.0,
      "so2": 0.74,
      "pm2_5": 9.62,
      "pm10": 9.99,
      "us-epa-index": 1,
      "gb-defra-index": 1
   },
   "alerts": [],
   "forecast": [
      {
         "date": "2025-09-17",
         "maxtemp_c": 19.6,
         "mintemp_c": 10.5,
         "condition": "Mist"
      },
      {
         "date": "2025-09-18",
         "maxtemp_c": 17.4,
         "mintemp_c": 14.1,
         "condition": "Overcast "
      },
      {
       

## Sending the Complete weather data to the Event Hub

In [0]:
import requests
import json
from azure.eventhub import EventHubProducerClient, EventHubConsumerClient, EventData
from datetime import datetime, timedelta

# Event Hub Configuration
EVENT_HUB_CONNECTION_STRING = dbutils.secrets.get(scope="key-vault-secret-scope", key="eventhub-connection-string")
EVENT_HUB_NAME = "weatherstreamingeventhub"

# Initialize the Event Hub Producer
producer = EventHubProducerClient.from_connection_string(
    conn_str=EVENT_HUB_CONNECTION_STRING,
    eventhub_name=EVENT_HUB_NAME
)

# Function to send events to Event Hub
def send_events(events):
    event_data_batch = producer.create_batch()
    event_data_batch.add(EventData(json.dumps(events)))
    producer.send_batch(event_data_batch)

# Function to handle the API response
def handle_response(response):
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code}, {response.text}"

# Function to get current weather and air quality data
def get_current_weather(base_url, api_key, location):
    current_weather_url = f"{base_url}/current.json"
    params = {
        'key': api_key,
        'q': location,
        "aqi": 'yes'
    }
    response = requests.get(current_weather_url, params=params)
    return handle_response(response)

# Function to get historical weather and air quality data
def get_history_weather(base_url, api_key, location, date):
    history_weather_url = f"{base_url}/history.json"
    params = {
        'key': api_key,
        'q': location,
        'dt': date,
        "aqi": 'yes'
    }
    response = requests.get(history_weather_url, params=params)
    return handle_response(response)

# Function to get Forecast Data
def get_forecast_weather(base_url, api_key, location, days):
    forecast_url = f"{base_url}/forecast.json"
    params = {
        "key": api_key,
        "q": location,
        "days": days,
    }
    response = requests.get(forecast_url, params=params)
    return handle_response(response)

# Function to get Alerts
def get_alerts(base_url, api_key, location):
    alerts_url = f"{base_url}/alerts.json"
    params = {
        'key': api_key,
        'q': location,
        "alerts": 'yes'
    }
    response = requests.get(alerts_url, params=params)
    return handle_response(response)

# Flatten and merge the data
def flatten_data(current_weather, forecast_weather, alerts, history_weather_list=None):
    location_data = current_weather.get("location", {})
    current = current_weather.get("current", {})
    condition = current.get("condition", {})
    air_quality = current.get("air_quality", {})
    forecast = forecast_weather.get("forecast", {}).get("forecastday", [])
    alert_list = alerts.get("alerts", {}).get("alert", [])
    history = []

    # Collect history from multiple days
    if history_weather_list:
        for hw in history_weather_list:
            days = hw.get("forecast", {}).get("forecastday", [])
            for day in days:
                history.append({
                    'date': day.get('date'),
                    'maxtemp_c': day.get('day', {}).get('maxtemp_c'),
                    'mintemp_c': day.get('day', {}).get('mintemp_c'),
                    'condition': day.get('day', {}).get('condition', {}).get('text')
                })

    flattened_data = {
        'name': location_data.get('name'),
        'region': location_data.get('region'),
        'country': location_data.get('country'),
        'lat': location_data.get('lat'),
        'lon': location_data.get('lon'),
        'localtime': location_data.get('localtime'),
        'temp_c': current.get('temp_c'),
        'is_day': current.get('is_day'),
        'condition_text': condition.get('text'),
        'condition_icon': condition.get('icon'),
        'wind_kph': current.get('wind_kph'),
        'wind_degree': current.get('wind_degree'),
        'wind_dir': current.get('wind_dir'),
        'pressure_in': current.get('pressure_in'),
        'precip_in': current.get('precip_in'),
        'humidity': current.get('humidity'),
        'cloud': current.get('cloud'),
        'feelslike_c': current.get('feelslike_c'),
        'uv': current.get('uv'),
        'air_quality': {
            'co': air_quality.get('co'),
            'no2': air_quality.get('no2'),
            'o3': air_quality.get('o3'),
            'so2': air_quality.get('so2'),
            'pm2_5': air_quality.get('pm2_5'),
            'pm10': air_quality.get('pm10'),
            'us-epa-index': air_quality.get('us-epa-index'),
            'gb-defra-index': air_quality.get('gb-defra-index')
        },
        'alerts': [
            {
                'headline': alert.get('headline'),
                'severity': alert.get('severity'),
                'description': alert.get('desc'),
                'instruction': alert.get('instruction')
            }
            for alert in alert_list
        ],
        'forecast': [
            {
                'date': day.get('date'),
                'maxtemp_c': day.get('day', {}).get('maxtemp_c'),
                'mintemp_c': day.get('day', {}).get('mintemp_c'),
                'condition': day.get('day', {}).get('condition', {}).get('text')
            }
            for day in forecast
        ],
        'history': history
    }
    return flattened_data

# Main program
def fetch_weather_data():
    base_url = "http://api.weatherapi.com/v1"
    location = "Regina"  # Replace with your city
    weatherapikey = dbutils.secrets.get(scope="key-vault-secret-scope", key="weatherapikey")

    # Collect last 3 days history
    history_weather_list = []
    for i in range(1, 8):  # last 7 days
        date = (datetime.today() - timedelta(days=i)).strftime("%Y-%m-%d")
        hw = get_history_weather(base_url, weatherapikey, location, date)
        history_weather_list.append(hw)

    # Get data from API
    current_weather = get_current_weather(base_url, weatherapikey, location)
    forecast_weather = get_forecast_weather(base_url, weatherapikey, location, 7)
    alerts = get_alerts(base_url, weatherapikey, location)

    # Flatten and merge data
    merged_data = flatten_data(current_weather, forecast_weather, alerts, history_weather_list)
    
    #Sending the weather data to Event HUB
    send_events(merged_data)

# Calling the Main Program
fetch_weather_data()

## Sending the weather data in streaming fashion

In [0]:
import requests
import json
from azure.eventhub import EventHubProducerClient, EventHubConsumerClient, EventData
from datetime import datetime, timedelta

# Event Hub Configuration
EVENT_HUB_CONNECTION_STRING = dbutils.secrets.get(scope="key-vault-secret-scope", key="eventhub-connection-string")
EVENT_HUB_NAME = "weatherstreamingeventhub"

# Initialize the Event Hub Producer
producer = EventHubProducerClient.from_connection_string(
    conn_str=EVENT_HUB_CONNECTION_STRING,
    eventhub_name=EVENT_HUB_NAME
)

# Function to send events to Event Hub
def send_events(events):
    event_data_batch = producer.create_batch()
    event_data_batch.add(EventData(json.dumps(events)))
    producer.send_batch(event_data_batch)

# Function to handle the API response
def handle_response(response):
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code}, {response.text}"

# Function to get current weather and air quality data
def get_current_weather(base_url, api_key, location):
    current_weather_url = f"{base_url}/current.json"
    params = {
        'key': api_key,
        'q': location,
        "aqi": 'yes'
    }
    response = requests.get(current_weather_url, params=params)
    return handle_response(response)

# Function to get historical weather and air quality data
def get_history_weather(base_url, api_key, location, date):
    history_weather_url = f"{base_url}/history.json"
    params = {
        'key': api_key,
        'q': location,
        'dt': date,
        "aqi": 'yes'
    }
    response = requests.get(history_weather_url, params=params)
    return handle_response(response)

# Function to get Forecast Data
def get_forecast_weather(base_url, api_key, location, days):
    forecast_url = f"{base_url}/forecast.json"
    params = {
        "key": api_key,
        "q": location,
        "days": days,
    }
    response = requests.get(forecast_url, params=params)
    return handle_response(response)

# Function to get Alerts
def get_alerts(base_url, api_key, location):
    alerts_url = f"{base_url}/alerts.json"
    params = {
        'key': api_key,
        'q': location,
        "alerts": 'yes'
    }
    response = requests.get(alerts_url, params=params)
    return handle_response(response)

# Flatten and merge the data
def flatten_data(current_weather, forecast_weather, alerts, history_weather_list=None):
    location_data = current_weather.get("location", {})
    current = current_weather.get("current", {})
    condition = current.get("condition", {})
    air_quality = current.get("air_quality", {})
    forecast = forecast_weather.get("forecast", {}).get("forecastday", [])
    alert_list = alerts.get("alerts", {}).get("alert", [])
    history = []

    # Collect history from multiple days
    if history_weather_list:
        for hw in history_weather_list:
            days = hw.get("forecast", {}).get("forecastday", [])
            for day in days:
                history.append({
                    'date': day.get('date'),
                    'maxtemp_c': day.get('day', {}).get('maxtemp_c'),
                    'mintemp_c': day.get('day', {}).get('mintemp_c'),
                    'condition': day.get('day', {}).get('condition', {}).get('text')
                })

    flattened_data = {
        'name': location_data.get('name'),
        'region': location_data.get('region'),
        'country': location_data.get('country'),
        'lat': location_data.get('lat'),
        'lon': location_data.get('lon'),
        'localtime': location_data.get('localtime'),
        'temp_c': current.get('temp_c'),
        'is_day': current.get('is_day'),
        'condition_text': condition.get('text'),
        'condition_icon': condition.get('icon'),
        'wind_kph': current.get('wind_kph'),
        'wind_degree': current.get('wind_degree'),
        'wind_dir': current.get('wind_dir'),
        'pressure_in': current.get('pressure_in'),
        'precip_in': current.get('precip_in'),
        'humidity': current.get('humidity'),
        'cloud': current.get('cloud'),
        'feelslike_c': current.get('feelslike_c'),
        'uv': current.get('uv'),
        'air_quality': {
            'co': air_quality.get('co'),
            'no2': air_quality.get('no2'),
            'o3': air_quality.get('o3'),
            'so2': air_quality.get('so2'),
            'pm2_5': air_quality.get('pm2_5'),
            'pm10': air_quality.get('pm10'),
            'us-epa-index': air_quality.get('us-epa-index'),
            'gb-defra-index': air_quality.get('gb-defra-index')
        },
        'alerts': [
            {
                'headline': alert.get('headline'),
                'severity': alert.get('severity'),
                'description': alert.get('desc'),
                'instruction': alert.get('instruction')
            }
            for alert in alert_list
        ],
        'forecast': [
            {
                'date': day.get('date'),
                'maxtemp_c': day.get('day', {}).get('maxtemp_c'),
                'mintemp_c': day.get('day', {}).get('mintemp_c'),
                'condition': day.get('day', {}).get('condition', {}).get('text')
            }
            for day in forecast
        ],
        'history': history
    }
    return flattened_data

def fetch_weather_data():
    base_url = "http://api.weatherapi.com/v1"
    location = "Regina"  # Replace with your city
    weatherapikey = dbutils.secrets.get(scope="key-vault-secret-scope", key="weatherapikey")

    # Collect last 3 days history
    history_weather_list = []
    for i in range(1, 8):  # last 7 days
        date = (datetime.today() - timedelta(days=i)).strftime("%Y-%m-%d")
        hw = get_history_weather(base_url, weatherapikey, location, date)
        history_weather_list.append(hw)

    # Get data from API
    current_weather = get_current_weather(base_url, weatherapikey, location)
    forecast_weather = get_forecast_weather(base_url, weatherapikey, location, 7)
    alerts = get_alerts(base_url, weatherapikey, location)

    # Flatten and merge data
    merged_data = flatten_data(current_weather, forecast_weather, alerts, history_weather_list)
    return merged_data

# Main program
def process_batch(batch_df, batch_id):
    try:     
        # Fetch weather data
        weather_data = fetch_weather_data()
        
        # Send the weather data (current weather part)
        send_events(weather_data)

    except Exception as e:
        print(f"Error sending events in batch {batch_id}: {str(e)}")
        raise e

# Set up a streaming source (for example, rate source for testing purposes)
streaming_df = spark.readStream.format("rate").option("rowsPerSecond", 1).load()

# Write the streaming data using foreachBatch to send weather data to Event Hub
query = streaming_df.writeStream.foreachBatch(process_batch).start()

query.awaitTermination()

# Close the producer after termination
producer.close()

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

## Sending the weather data to Event Hub Every 1 min

In [0]:
import requests
import json
from azure.eventhub import EventHubProducerClient, EventHubConsumerClient, EventData
from datetime import datetime, timedelta

# Event Hub Configuration
EVENT_HUB_CONNECTION_STRING = dbutils.secrets.get(scope="key-vault-secret-scope", key="eventhub-connection-string")
EVENT_HUB_NAME = "weatherstreamingeventhub"

# Initialize the Event Hub Producer
producer = EventHubProducerClient.from_connection_string(
    conn_str=EVENT_HUB_CONNECTION_STRING,
    eventhub_name=EVENT_HUB_NAME
)

# Function to send events to Event Hub
def send_events(events):
    event_data_batch = producer.create_batch()
    event_data_batch.add(EventData(json.dumps(events)))
    producer.send_batch(event_data_batch)

# Function to handle the API response
def handle_response(response):
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code}, {response.text}"

# Function to get current weather and air quality data
def get_current_weather(base_url, api_key, location):
    current_weather_url = f"{base_url}/current.json"
    params = {
        'key': api_key,
        'q': location,
        "aqi": 'yes'
    }
    response = requests.get(current_weather_url, params=params)
    return handle_response(response)

# Function to get historical weather and air quality data
def get_history_weather(base_url, api_key, location, date):
    history_weather_url = f"{base_url}/history.json"
    params = {
        'key': api_key,
        'q': location,
        'dt': date,
        "aqi": 'yes'
    }
    response = requests.get(history_weather_url, params=params)
    return handle_response(response)

# Function to get Forecast Data
def get_forecast_weather(base_url, api_key, location, days):
    forecast_url = f"{base_url}/forecast.json"
    params = {
        "key": api_key,
        "q": location,
        "days": days,
    }
    response = requests.get(forecast_url, params=params)
    return handle_response(response)

# Function to get Alerts
def get_alerts(base_url, api_key, location):
    alerts_url = f"{base_url}/alerts.json"
    params = {
        'key': api_key,
        'q': location,
        "alerts": 'yes'
    }
    response = requests.get(alerts_url, params=params)
    return handle_response(response)

# Flatten and merge the data
def flatten_data(current_weather, forecast_weather, alerts, history_weather_list=None):
    location_data = current_weather.get("location", {})
    current = current_weather.get("current", {})
    condition = current.get("condition", {})
    air_quality = current.get("air_quality", {})
    forecast = forecast_weather.get("forecast", {}).get("forecastday", [])
    alert_list = alerts.get("alerts", {}).get("alert", [])
    history = []

    # Collect history from multiple days
    if history_weather_list:
        for hw in history_weather_list:
            days = hw.get("forecast", {}).get("forecastday", [])
            for day in days:
                history.append({
                    'date': day.get('date'),
                    'maxtemp_c': day.get('day', {}).get('maxtemp_c'),
                    'mintemp_c': day.get('day', {}).get('mintemp_c'),
                    'condition': day.get('day', {}).get('condition', {}).get('text')
                })

    flattened_data = {
        'name': location_data.get('name'),
        'region': location_data.get('region'),
        'country': location_data.get('country'),
        'lat': location_data.get('lat'),
        'lon': location_data.get('lon'),
        'localtime': location_data.get('localtime'),
        'temp_c': current.get('temp_c'),
        'is_day': current.get('is_day'),
        'condition_text': condition.get('text'),
        'condition_icon': condition.get('icon'),
        'wind_kph': current.get('wind_kph'),
        'wind_degree': current.get('wind_degree'),
        'wind_dir': current.get('wind_dir'),
        'pressure_in': current.get('pressure_in'),
        'precip_in': current.get('precip_in'),
        'humidity': current.get('humidity'),
        'cloud': current.get('cloud'),
        'feelslike_c': current.get('feelslike_c'),
        'uv': current.get('uv'),
        'air_quality': {
            'co': air_quality.get('co'),
            'no2': air_quality.get('no2'),
            'o3': air_quality.get('o3'),
            'so2': air_quality.get('so2'),
            'pm2_5': air_quality.get('pm2_5'),
            'pm10': air_quality.get('pm10'),
            'us-epa-index': air_quality.get('us-epa-index'),
            'gb-defra-index': air_quality.get('gb-defra-index')
        },
        'alerts': [
            {
                'headline': alert.get('headline'),
                'severity': alert.get('severity'),
                'description': alert.get('desc'),
                'instruction': alert.get('instruction')
            }
            for alert in alert_list
        ],
        'forecast': [
            {
                'date': day.get('date'),
                'maxtemp_c': day.get('day', {}).get('maxtemp_c'),
                'mintemp_c': day.get('day', {}).get('mintemp_c'),
                'condition': day.get('day', {}).get('condition', {}).get('text')
            }
            for day in forecast
        ],
        'history': history
    }
    return flattened_data

def fetch_weather_data():
    base_url = "http://api.weatherapi.com/v1"
    location = "Regina"  # Replace with your city
    weatherapikey = dbutils.secrets.get(scope="key-vault-secret-scope", key="weatherapikey")

    # Collect last 3 days history
    history_weather_list = []
    for i in range(1, 31):  # last 30 days
        date = (datetime.today() - timedelta(days=i)).strftime("%Y-%m-%d")
        hw = get_history_weather(base_url, weatherapikey, location, date)
        history_weather_list.append(hw)

    # Get data from API
    current_weather = get_current_weather(base_url, weatherapikey, location)
    forecast_weather = get_forecast_weather(base_url, weatherapikey, location, 8)
    alerts = get_alerts(base_url, weatherapikey, location)

    # Flatten and merge data
    merged_data = flatten_data(current_weather, forecast_weather, alerts, history_weather_list)
    return merged_data

last_sent_time = datetime.now() - timedelta(seconds=60)  # Initial value

# Main program
def process_batch(batch_df, batch_id):
    global last_sent_time
    try:
        # Get current time
        current_time = datetime.now()
        
        # Check if 30 seconds have passed since last event was sent
        if (current_time - last_sent_time).total_seconds() >= 60:
            # Fetch weather data
            weather_data = fetch_weather_data()
            
            # Send the weather data (current weather part)
            send_events(weather_data)

            # Update last sent time
            last_sent_time = current_time
            print(f'Event Sent at {last_sent_time}')

    except Exception as e:
        print(f"Error sending events in batch {batch_id}: {str(e)}")
        raise e

# Set up a streaming source (for example, rate source for testing purposes)
streaming_df = spark.readStream.format("rate").option("rowsPerSecond", 1).load()

# Write the streaming data using foreachBatch to send weather data to Event Hub
query = streaming_df.writeStream.foreachBatch(process_batch).start()

query.awaitTermination()

# Close the producer after termination
producer.close()

Event Sent at 2025-09-17 23:38:42.430805
Event Sent at 2025-09-17 23:39:43.211241
Event Sent at 2025-09-17 23:40:44.497200


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can